Парсер новостей с сайта ФСБ

Последние изменения кода: 2022-02-10

## Библиотеки и настройки

In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = 'C:/00_Data/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 9


In [6]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

Номер прокси, от 0 до 8
0


In [7]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
#opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [8]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

Login: tw1QwuaQ
Pass: EJ1SMvdd


In [9]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

In [ ]:
=&PAGEN_1=4

## Собираю ссылки на новости

Сайт немного кривой и тормознутый. Поэтому в коде длинные задержки.

In [13]:
start_url = 'https://culture.gov.ru/documents/?DOCS%5BKEYWORDS%5D=исключение&DOCS%5BVIEW_DOCUMENTS%5D=&DOCS%5BAUTHORITY%5D=&DOCS%5BTYPE_DOCUMENTS%5D=&DOCS%5BDATE_1%5D=&DOCS%5BDATE_2%5D=&DOCS%5BNUMBER%5D=&PAGEN_1='

In [36]:
data = [] # список для данных

for i in tqdm(range(1, 311)):
    
    driver.get(start_url + str(i))
    sleep(5)
    
    
    doc_blocks = driver.find_elements(By.CLASS_NAME, 'b-material__item')
    
    for document in doc_blocks:
        doc_name = document.find_element(By.TAG_NAME, 'a').text
        doc_url = document.find_element(By.TAG_NAME, 'a').get_attribute('href')
        doc_date = document.find_element(By.CLASS_NAME, 'b-article__date').text
        
        data.append([doc_name, doc_date, doc_url])

  0%|          | 0/310 [00:00<?, ?it/s]

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00677AC3+2587331]
	Ordinal0 [0x0060ADD1+2141649]
	Ordinal0 [0x00503BB8+1063864]
	Ordinal0 [0x004F58B7+1005751]
	Ordinal0 [0x004F48B7+1001655]
	Ordinal0 [0x004F4C87+1002631]
	Ordinal0 [0x004FF0AF+1044655]
	Ordinal0 [0x005098EB+1087723]
	Ordinal0 [0x0050C000+1097728]
	Ordinal0 [0x004F4F86+1003398]
	Ordinal0 [0x00509795+1087381]
	Ordinal0 [0x00558900+1411328]
	Ordinal0 [0x005484A6+1344678]
	Ordinal0 [0x005253F6+1201142]
	Ordinal0 [0x005262E6+1204966]
	GetHandleVerifier [0x0081DF22+1680738]
	GetHandleVerifier [0x008D0DBC+2413564]
	GetHandleVerifier [0x0070D151+563089]
	GetHandleVerifier [0x0070BF13+558419]
	Ordinal0 [0x0061081E+2164766]
	Ordinal0 [0x00615508+2184456]
	Ordinal0 [0x00615650+2184784]
	Ordinal0 [0x0061F5BC+2225596]
	BaseThreadInitThunk [0x7662FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D67A7E+286]
	RtlGetAppConta

In [33]:
# преобразовываю в датафрейм

head = ['doc_name', 'doc_date', 'doc_url']

work_urls = pd.DataFrame(data, columns=head)

In [34]:
work_urls.shape

(2, 3)

In [35]:
work_urls

,doc_name,doc_date,doc_url
0,Секция станковой (масляной) живописи Аттестаци...,08 ФЕВРАЛЯ 2022,https://culture.gov.ru/documents/sektsiya-stan...
1,Акт государственной историко-культурной экспер...,31 ЯНВАРЯ 2022,https://culture.gov.ru/documents/akt-gosudarst...


In [ ]:
work_data = work_data.merge(work_urls, how='left', on='news_url')

In [ ]:
# Записываю в файл

work_data.to_csv(file_path + 'news.csv', encoding='utf-8', index=False)

In [ ]:
driver.quit()